In [51]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from copy import deepcopy
# from math import factorial, exp

In [52]:
%run "attack_graph_MARA.ipynb"

## Pre-Processing

Please note this difference:
Adding virtual start node: We ADD a new virtual starting node and connect it to existing root (starting nodes)
Adding virtual target node: we REMOVE the old target nodes and replace them with one joint virtual target node

### Add Virtual starting node

In [53]:
def find_and_add_entry_node(graph):
    # Find all nodes with no incoming edges
    roots = [n for n, deg in graph.in_degree() if deg == 0]
    #print("DEBUG: Roots before adding entry node:", roots)
    
    if len(roots) > 1:
        # Create a virtual entry node
        entry = "attacker_entry_node"
        graph.add_node(entry)
        
        # Connect the entry node to all roots
        for r in roots:
            graph.add_edge(entry, r, weight=1)
        
        # print("DEBUG: Multiple roots found. Created entry node:", entry)
        # print("DEBUG: Edges from entry node to roots:")
        # for r in roots:
        #     print(f"  {entry} -> {r}")
        # return entry, graph
    else:
        # If single root exists, that is our entry
        entry = roots[0]
        #print("DEBUG: Only one root found. Entry node is:", entry)
        return entry, graph


### Add Virtual Target Node 

In [54]:
def process_target_nodes(graph):
    # In R:
    # jointVertex <- gorder(attack_graph) - length(target_list) + 1
    # vertexNo[, target_list] <- jointVertex
    # vertexNo[vertexNo == 0] <- 1:(jointVertex - 1)
    # attack_graph <- contract.vertices(attack_graph, mapping = vertexNo)

    # Step 1: Identify target nodes (no outgoing edges)
    targets = [n for n, deg in graph.out_degree() if deg == 0]
    if len(targets) <= 1:
        # If there is 0 or 1 target, no merging needed
        return graph

    # Number of original nodes
    original_count = graph.number_of_nodes()
    # same formula as in the R code
    joint_vertex_num = original_count - len(targets) + 1

    # We will mimic exactly the logic from R:
    # First, get a topological order of the original graph
    topo_nodes = list(nx.topological_sort(graph))

    # Create a mapping array: For each node in topo order, assign a "vertexNo".
    # Initialize everything to 0
    vertexNo = {node: 0 for node in topo_nodes}

    # Set all targets to joint_vertex_num
    for t in targets:
        vertexNo[t] = joint_vertex_num

    # Now fill the zeros with consecutive integers from 1 to joint_vertex_num-1
    # but maintain the topological order
    fill_value = 1
    for node in topo_nodes:
        if vertexNo[node] == 0:
            vertexNo[node] = fill_value
            fill_value += 1

    # Now we have a mapping of each original node to an integer ID.
    # Nodes assigned the same integer ID should be contracted into one node.
    # Example: If multiple targets got joint_vertex_num, they form one merged node.

    # Create a mapping from the integer IDs to sets of original nodes
    block_map = {}
    for node, grp_id in vertexNo.items():
        if grp_id not in block_map:
            block_map[grp_id] = set()
        block_map[grp_id].add(node)

    # Build a new contracted graph
    merged_graph = nx.DiGraph()

    # Add nodes (each group_id becomes one node in merged_graph)
    for grp_id in block_map:
        merged_graph.add_node(grp_id)

    # For edges: if u->v in the old graph, map them to their groups:
    # If they map to the same group, skip (self-loop)
    # Otherwise, add edge between the groups
    for u, v, data in graph.edges(data=True):
        u_id = vertexNo[u]
        v_id = vertexNo[v]
        if u_id != v_id:
            # Keep the weight as is
            w = data.get('weight', 1)
            if not merged_graph.has_edge(u_id, v_id):
                merged_graph.add_edge(u_id, v_id, weight=w)

    return merged_graph

In [55]:
# Prepare the graph
work_graph = deepcopy(attack_graph)
entry_node, work_graph = find_and_add_entry_node(work_graph)

work_graph = process_target_nodes(work_graph)

## Static elements preparation for the Game

### Generate game elements

In [56]:
# This method:
# Finds single target node in graph (after merging)
# Gets all possible paths from entry to target node
# Creates a sorted list of all (unique, no duplicates) nodes in any path
# Creates a topologically sorted order of these nodes
# Determines nodes where defender can act (excluding entry/target/roots)

# Returns:

# routes: List of all possible attack paths
# V: List of all nodes involved in any path
# as1: List of nodes where defender can act
# as2: Same as routes (all attack paths)
# target_list: List containing single target node
# node_order: Topologically sorted list of nodes

def generate_game_elements(graph, entry_node):
    """
    Generate game elements after we have the final graph (with single entry and single target).
    """
    # determine the target node
    # Important note: This code assumes that we have already merged all target nodes into one
    target_list = [n for n,d in graph.out_degree() if d == 0]
    if len(target_list) != 1:
        print("WARNING: Expected exactly one target node after contraction. Found:", target_list)
    
    # Find all simple paths from entry to target
    routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))

    # Flatten the routes that we found and sort them 
    # i.e. list of nodes where attacker could be (+ start/end node)
    V = sorted(list(set([node for path in routes for node in path])))

    # This gives us nodes in order where each node comes after its predecessors
    topo_all = list(nx.topological_sort(graph))
    # Same as topo_all in this case since all nodes are in V
    node_order = [n for n in topo_all if n in V]
    
    # prepare nodes to exclude (start and target node)
    excluded = {entry_node} | set(target_list)
    # if we have multiple roots, exclude them as well as 
    # attacker can't be at: virtual entry node, normal entry node, target node or virtual target node
    if entry_node == "attacker_entry_node":
        roots = [r for r, deg in graph.in_degree() if deg == 0 and r != entry_node]
        excluded.update(roots)
    
    # as1 = [2,3,5] - these are the nodes where defender can act
    as1 = [n for n in V if n not in excluded]
    # as2 are the routes that attacker can take
    as2 = routes
    
    return routes, V, as1, as2, target_list, node_order

In [57]:
# Creates a list of all possible attacker locations (excluding entry and target nodes)
# Assigns equal probability (1/n) to each possible location in a dictionary
# Counts the total number of attack paths

# Returns:
# adv_list: List of nodes where attacker could be
# theta: Dictionary mapping each possible location to its probability (all equal)
# m: Number of attack paths

def setup_game_parameters(V, routes, entry_node, target_list):
    # prepare all the possible locations of attacker (avatars)
    adv_list = [n for n in V if n not in [entry_node] + target_list]
    if len(adv_list) == 0:
        print("WARNING: No adversary intermediate locations found. Check graph structure.")
    
    # Create a dictionary that assigns equal probability to each possible attacker location
    theta = {loc: 1/len(adv_list) for loc in adv_list}
    # Number of attack paths
    m = len(routes)
    return adv_list, theta, m

## Dynamic elements preparation for the Game

### Method to Calculate the Pay_Offs 

#### Some Explanations

In [ ]:
# This method returns "payoffs":
# payoffs = [
#     {'dpdf': [0.1, 0.2, 0.7], 'support': [1,2,3], ...},  # for check1 + path1
#     {'dpdf': [0.3, 0.5, 0.2], 'support': [1,2,3], ...},  # for check1 + path2
#     {'dpdf': [0.4, 0.4, 0.2], 'support': [1,2,3], ...},  # for check2 + path1
#     {'dpdf': [0.1, 0.1, 0.8], 'support': [1,2,3], ...}   # for check2 + path2
# ]

Let's look at:
{'dpdf': [0.1, 0.2, 0.7], 'support': [1,2,3]}  # for check1 + path1

This means:

When defender checks at location 1, and attacker uses path1:

10% chance attacker is at first node (0.1)

20% chance attacker is at second node (0.2)

70% chance attacker reached the target node! (0.7)

Note: Support [1,2,3] are NOT the node number. I think of them as "steps away from start" rather than actual node IDs.

That's why we take dpdf[-1] for our payoff matrix - it's the probability the attacker successfully reaches the target node under this check+path combination.

##### Method necessary to calc. Pay offs

In [58]:
def lossDistribution(U):
    U = U / np.sum(U)
    support = np.arange(1, len(U)+1)
    dpdf = U
    cdf = np.cumsum(U)
    tail = 1 - cdf + U
    return {
        'support': support,
        'dpdf': dpdf,
        'cdf': cdf,
        'tail': tail
    }

#### End of Explanations

In [59]:
def calculate_payoff_distribution(graph, as1, as2, V, adv_list, theta, random_steps_fn, 
                                  attack_rate, defense_rate, node_order):
    """
    graph: The attack graph
    as1: List of nodes where the defender checks
    as2: List of attack paths
    V: List of all nodes in the game (topological order, taken as unique nodes across all attack paths)
    adv_list: List of potential attacker start locations (V - {entry, target} nodes)
    theta: Defenders belief where the attacker might start via weights (if unknown then just uniform 1/|adv_list|)
    random_steps_fn: Function that returns probability distribution for number of steps attacker can take
    attack_rate: Lambda parameter controlling attacker's movement speed (e.g., 2 moves per time unit)
    defense_rate: Lambda parameter controlling defender's check frequency (e.g., 1 check per time unit)
    node_order: Topological sorting of nodes to ensure consistent ordering in probability distributions
    
    Returns:
    List of payoff distributions, one for each combination of defender check location × attack path.
    Each distribution shows likelihood of attacker being at different nodes, with final value being
    probability of reaching target.
    """

    
    payoffs = []
    for i in as1: # where defender checks 
        for path in as2: # attack path for attacker
            U = np.zeros(len(V))
            for avatar in adv_list:
                L = np.zeros(len(V))
                if avatar in path:
                    start_idx = path.index(avatar)
                    route = path[start_idx:]
                    pdf_d = random_steps_fn(route, attack_rate, defense_rate)
                    
                    if i in route:
                        cutPoint = route.index(i) + 1
                    else:
                        cutPoint = len(route)
                    
                    if np.sum(pdf_d[:cutPoint]) == 0:
                        payoffDistr = np.zeros(cutPoint)
                        payoffDistr[-1] = 1.0
                    else:
                        payoffDistr = pdf_d[:cutPoint] / np.sum(pdf_d[:cutPoint])
                    
                    for idx, node in enumerate(route[:cutPoint]):
                        L[V.index(node)] = payoffDistr[idx]
                else:
                    L[V.index(avatar)] = 1.0
                
                U += theta[avatar] * L
            
            if np.sum(U) == 0:
                U = np.full_like(U, 1e-7)
            else:
                U /= np.sum(U)
            
            node_positions = [V.index(n) for n in node_order]
            U = U[node_positions]
            U[U == 0] = 1e-7
            ld = lossDistribution(U)
            payoffs.append(ld)

    return payoffs

### Method for finding optimal solutions

#### Some Explanations

This method solves a zero-sum game using linear programming. It first creates a proper payoff matrix from the loss distributions, then solves two LPs: one for defender minimizing attacker success probability, and one for attacker maximizing it. Both optimizations should yield the same equilibrium value.

Constructs payoff matrix
Solves LP for defender FIRST (minimizing attacker success)
Then solves LP for attacker (maximizing their success)
Verifies both values match (equilibrium property)

The method returns a dictionary: 

{

    'optimal_defense': {3: 0.6, 4: 0.4},  # defend node 3 with 60% probability, node 4 with 40%                     
    'attacker_strategy': [0.3, 0.5, 0.2], # use path 1 with 30% probability, path 2 with 50%, etc.
    'defender_success': 0.128,           # defender can keep attacker success ≤ 0.128              
    'attacker_success': 0.128            # attacker can achieve at least 0.128
                                            
}

#### End of Explanations

In [48]:
def solve_game(payoffs, as1, as2):
    n = len(as1)
    m = len(as2)
    
    # Create payoff matrix
    payoff_matrix = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            idx = i*m + j
            ld = payoffs[idx]
            payoff_matrix[i, j] = ld['dpdf'][-1]
    
    ### Start Defender's optimization ###
    c = np.zeros(n+1)
    c[0] = 1.0
    
    A_ub = np.zeros((m, n+1))
    b_ub = np.zeros(m)
    for j in range(m):
        A_ub[j,0] = -1.0
        for i in range(n):
            A_ub[j,i+1] = payoff_matrix[i,j]
            
    A_eq = np.zeros((1, n+1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])
    
    bounds = [(0,None)]*(n+1)
    
    v_defender = None
    v_attacker = None
    
    # Solve the LP
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    
    ### End Defender's optimization ###
    if res.success:
        # Extract the results for later logging
        v_defender = res.x[0]
        x_def = res.x[1:]
        
        ### Start Attacker's optimization ###
        c_att = np.zeros(m+1)
        c_att[0] = -1.0
        
        A_ub_att = np.zeros((n, m+1))
        b_ub_att = np.zeros(n)
        for i in range(n):
            A_ub_att[i,0] = 1.0
            for j in range(m):
                A_ub_att[i,j+1] = -payoff_matrix[i,j]
                
        A_eq_att = np.zeros((1, m+1))
        A_eq_att[0,1:] = 1.0
        b_eq_att = np.array([1.0])
        
        bounds_att = [(0,None)]*(m+1)
        res_att = linprog(c_att, A_ub=A_ub_att, b_ub=b_ub_att, 
                         A_eq=A_eq_att, b_eq=b_eq_att, bounds=bounds_att)
        
        ### End Attacker's optimization ###
        
        if res_att.success:
            # Extract attacker results for later logging
            y_att = res_att.x[1:]
            v_attacker = res_att.x[0]   # new - remove the negative sign because c_att[0] = -1.0
            
            # Now both values are defined, we can check
            if abs(v_defender - v_attacker) > 1e-5:
                logger.info("\nWarning: Defender and attacker values don't match!")
                logger.info(f"Defender value: {v_defender:.6f}")
                logger.info(f"Attacker value: {v_attacker:.6f}")
            
            return {
                'optimal_defense': dict(zip(as1, x_def)),
                'attacker_strategy': y_att,
                'defender_success': v_defender,
                'attacker_success': v_attacker
            }
    
    logger.info("LP optimization failed")
    return None

### Method to run the Actual Game

#### Some Explanations

Key Steps:

1. Finds entry node in graph
2. Generates game elements (routes, nodes, defense locations) using generate_game_elements()
3. Sets up game parameters (adversary locations, probabilities) using setup_game_parameters()

Then For each attack/defense rate combination:

5. Calculates payoff distributions
6. Solves game using solve_game()
7. Logs results (optimal strategies, success probabilities)

Returns:
Nothing - this method only logs results to the logger
All output goes to log file 

#### End of Explanations

In [49]:
def run_game(graph, attack_rate_list, defense_rate_list, random_steps_fn):
    entry_node_candidates = [n for n,deg in graph.in_degree() if deg==0]
    if len(entry_node_candidates) != 1:
        print("WARNING: Not a single entry node found.")
    entry_node = entry_node_candidates[0]

    routes, V, as1, as2, target_list, node_order = generate_game_elements(graph, entry_node)
    adv_list, theta, m = setup_game_parameters(V, routes, entry_node, target_list)
    
    if not defense_rate_list:
        defense_rate_list = [0]
    if not attack_rate_list:
        attack_rate_list = [0]
    
    for defenseRate in defense_rate_list:
        for attackRate in attack_rate_list:
            logger.info("\n++++++++++++++++++++++++++++++++")
            logger.info(f"attack rate =  {attackRate} , defense rate =  {defenseRate} \n")
            logger.info("\tequilibrium for multiobjective security game (MOSG)\n")
            
            payoffs = calculate_payoff_distribution(
                graph, as1, as2, V, adv_list, theta, random_steps_fn, attackRate, defenseRate, node_order
            )
            
            eq = solve_game(payoffs, as1, as2)
            if eq is not None:
                logger.info("optimal defense strategy:")
                logger.info("         prob.")
                for node, prob in sorted(eq['optimal_defense'].items(), key=lambda x: str(x[0])):
                    logger.info(f"{node} {prob:.6e}")
                
                logger.info("\nworst case attack strategies per goal:")
                logger.info("          1")
                if 'attacker_strategy' in eq:
                    for idx, prob in enumerate(eq['attacker_strategy'], 1):
                        logger.info(f"{idx} {prob:.7f}")
                logger.info(f"[1] {eq['attacker_success']:.3f}")
                
                # Add these two lines for the values
                logger.info(f"\nDefender can keep attacker success below: {eq['defender_success']:.3f}")
                logger.info(f"Attacker can guarantee success probability of: {eq['attacker_success']:.3f}")

### Method to be called in different Notebook

In [38]:
# This method is supposed to be run in experiment_X.ipynb files. It is not a standalone script.
def main():
    run_game(work_graph, attack_rate_list=attack_rate_list, defense_rate_list=defense_rate_list, random_steps_fn=random_steps)